In [13]:
import pandas as pd
import numpy as np
import json

with open('errores_ortograficos.json', "r", encoding= "utf-8") as f:
    errores_ortograficos = json.load(f)
errores_ortograficos

[{'palabra': 'que', 'frecuencia': 444, 'errores_ortograficos': ['q']},
 {'palabra': 'matricula', 'frecuencia': 350, 'errores_ortograficos': []},
 {'palabra': 'puedo', 'frecuencia': 257, 'errores_ortograficos': []},
 {'palabra': 'retiro', 'frecuencia': 211, 'errores_ortograficos': []},
 {'palabra': 'debo', 'frecuencia': 192, 'errores_ortograficos': []},
 {'palabra': 'solicitud',
  'frecuencia': 189,
  'errores_ortograficos': ['solisitud']},
 {'palabra': 'como', 'frecuencia': 188, 'errores_ortograficos': []},
 {'palabra': 'hacer', 'frecuencia': 156, 'errores_ortograficos': ['acer']},
 {'palabra': 'constancia',
  'frecuencia': 140,
  'errores_ortograficos': ['constansia']},
 {'palabra': 'ciclo', 'frecuencia': 129, 'errores_ortograficos': []},
 {'palabra': 'hola', 'frecuencia': 113, 'errores_ortograficos': ['ola']},
 {'palabra': 'autoseguro',
  'frecuencia': 112,
  'errores_ortograficos': ['auto-seguro', 'auto seguro']},
 {'palabra': 'curso', 'frecuencia': 110, 'errores_ortograficos': []},

In [14]:
word2miss = {}

for e in errores_ortograficos:
    word = e['palabra']
    misspellings = e['errores_ortograficos']
    word2miss[word] = misspellings
word2miss

{'que': ['q'],
 'matricula': [],
 'puedo': [],
 'retiro': [],
 'debo': [],
 'solicitud': ['solisitud'],
 'como': [],
 'hacer': ['acer'],
 'constancia': ['constansia'],
 'ciclo': [],
 'hola': ['ola'],
 'autoseguro': ['auto-seguro', 'auto seguro'],
 'curso': [],
 'total': [],
 'si': [],
 'quiero': [],
 'pagar': [],
 'modelo': [],
 'cuando': [],
 'parcial': [],
 'solicitar': ['solisitar'],
 'matricularme': [],
 'realizar': ['realisar'],
 'retirarme': [],
 'donde': [],
 'saber': [],
 'reserva': ['recerva', 'reserba', 'recerba'],
 'pago': [],
 'cual': [],
 'horario': [],
 'cursos': [],
 'buenas': [],
 'algun': [],
 'procedimiento': ['prosedimiento'],
 'enviar': [],
 'estudios': [],
 'tramite': [],
 'reincorporacion': ['reincorporasion', 're incorporacion'],
 'formato': [],
 'carnet': [],
 'dias': [],
 'fechas': [],
 'alumno': [],
 'notas': [],
 'quisiera': ['quiciera'],
 'tramitar': [],
 'mas': [],
 'semana': [],
 'fecha': [],
 'solo': [],
 'horarios': [],
 'rezagado': ['resagado'],
 'unive

In [69]:
##save
import json

with open('errores_ortograficos.json', "w", encoding= "utf-8") as f:
    json.dump(errores_ortograficos, f)

In [50]:
import random
import spacy
from nltk.corpus import stopwords

nlp = spacy.load('es_core_news_md')

## Obtener stopwords
stop_words = stopwords.words('spanish')

marks = ["?",".", ",","¿"]
stop_words.extend(marks)

question_words = ['qué','que','quién','quien', 'cuál','cual', 'dónde','donde', 'cuándo','cuando', 'cómo','como', 'por qué','por que','porqué','porque', 
'cuánto','cuanto','cuantos']
stop_words = [ sw for sw in stop_words if not sw in question_words ]

## tokenizar

def tokenize(text):
    tokens = [str(token) for token in nlp(text)]
    return tokens

## Generar stopwords
text = "cuando procede el retiro total de los cursos"

prob = 0.5
[misspellings_event] = random.choices([0,1], weights= [1-prob, prob], k = 1)

def insert_natural_misspelling(text):
    tokens = tokenize(text)
    idx2token  = { str(idx):token for idx, token in enumerate(tokens) if not token in stop_words}
    ## elegir palabra 
    idx = int(random.choice(list(idx2token.keys())))
    ## remplazar por error otrografico si hay 
    word = tokens[idx]
    print("Palabra seleccionada:", word)
    misspellings = word2miss[word]
    if len(misspellings) > 0:
        misspelling = random.choice(misspellings)
        tokens[idx] = misspelling
        new_text = " ".join(tokens)
        return new_text
    
    return None

with open('./keybord_es.json', 'r') as f:
    keyboard_misspelings = json.load(f)

class CharacterSwap:
    def __init__(self):
        pass

    def insert_misspeling(self, word, idx):
        chars = [*word]
        n_chars = len(word)
        if idx < n_chars - 1:
            temp_char = chars[idx]
            chars[idx] = chars[idx + 1]
            chars[idx + 1] = temp_char
        else:
            temp_char = chars[idx]
            chars[idx] = chars[idx - 1]
            chars[idx - 1] = temp_char
        return "".join(chars)

class CharacterDuplicate:
    def __init__(self):
        pass

    def insert_misspeling(self, word, idx):
        chars = [*word]
        n_chars = len(word)
        chars.insert(idx, chars[idx])
        return "".join(chars)

class CharacterDelete:
    def __init__(self):
        pass

    def insert_misspeling(self, word, idx):
        chars = [*word]
        n_chars = len(word)
        del chars[idx]
        return "".join(chars)


class CharacterKeyboard:
    def __init__(self):
        pass

    def insert_misspeling(self, word, idx):
        chars = [*word]
        n_chars = len(word)
        misspelings = keyboard_misspelings[chars[idx]]
        chars[idx] = random.choice(misspelings)
        return "".join(chars)


class CharacterMS:
    def __init__(self, char_max = 2, char_probs = [0.89,0.11]):
        self.char_max = char_max,
        self.char_probs = char_probs

    def get_idxes(self, word):
        [num_idxes]  = random.choices([1,2], weights= self.char_probs)
        self.chars = [*word]
        self.n_chars = len(word)
        idxes = random.sample(list(range(self.n_chars)), k = num_idxes)
        return idxes

    def generate_misspeling(self, word):
        idxs = self.get_idxes(word)
        ms_types = random.choices([1,2,3,4], k= len(idxs))
        for idx, ms_type in zip(idxs, ms_types):
            word = self.insert_misspelling(word, idx, ms_type)
        
        return word

    def insert_misspelling(self, word, idx, ms_type):
        if ms_type == 1:
            return CharacterSwap().insert_misspeling(word, idx)
        elif ms_type == 2:
            return CharacterDuplicate().insert_misspeling(word, idx)
        elif ms_type == 3:
            return CharacterDelete().insert_misspeling(word, idx)
        elif ms_type == 4:
            return CharacterKeyboard().insert_misspeling(word, idx)
    

def insert_sintetic_misspelling(text):
    tokens = tokenize(text)
    idx2token  = { str(idx):token for idx, token in enumerate(tokens) if not token in stop_words}
    ## elegir palabra 
    idx = int(random.choice(list(idx2token.keys())))
    ## remplazar por error otrografico si hay 
    word = tokens[idx]
    print("Palabra seleccionada:", word)
    CharacterMS()

#idx2token  = None
if misspellings_event == 1:
    print('Generar error ortografico')
    #new_text = insert_natural_misspelling(text)

Generar error ortografico
Palabra seleccionada: procede


'cuando prosede el retiro total de los cursos'

In [62]:

def generate_misspelling_swap(word):
    chars = [*word]
    n_chars = len(word)
    idx = random.choice(range(n_chars))
    if idx < n_chars - 1:
        temp_char = chars[idx]
        chars[idx] = chars[idx + 1]
        chars[idx + 1] = temp_char
    else:
        temp_char = chars[idx]
        chars[idx] = chars[idx - 1]
        chars[idx - 1] = temp_char
    return "".join(chars)
generate_misspelling_swap("hola")

'ohla'

In [ ]:
def generate_misspelling_swap(word):
    chars = [*word]
    n_chars = len(word)
    idx = random.choice(range(n_chars))
    if idx < n_chars - 1:
        temp_char = chars[idx]
        chars[idx] = chars[idx + 1]
        chars[idx + 1] = temp_char
    else:
        temp_char = chars[idx]
        chars[idx] = chars[idx - 1]
        chars[idx - 1] = temp_char
    return "".join(chars)

def generate_misspelling_duplicate(word):
    chars = [*word]
    n_chars = len(word)
    idx = random.choice(range(n_chars))
    chars.insert(idx, chars[idx])
    return "".join(chars)

def generate_misspelling_delete(word):
    chars = [*word]
    n_chars = len(word)
    idx = random.choice(range(n_chars))
    del chars[idx]
    return "".join(chars)

def generate_misspelling_keyboard(word):
    chars = [*word]
    n_chars = len(word)
    idx = random.choice(range(n_chars))
    misspelings = keyboard_misspelings[chars[idx]]
    chars[idx] = random.choice(misspelings)
    return "".join(chars)
